# Testing

##### The Design 단계 : Self-Corrective RAG

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


# --- Create an index of documents ---

urls = [
    "https://blog.langchain.dev/top-5-langgraph-agents-in-production-2024/",
    "https://blog.langchain.dev/langchain-state-of-ai-2024/",
    "https://blog.langchain.dev/introducing-ambient-agents/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

# Retrieve the relevant documents
results = retriever.invoke(
    "What are 2 LangGraph agents used in production in 2024?")

print("Results: \n", results)


# --- Create a grader for retrieved documents ---

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with structured output
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human",
         "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

# --- Grade retrieved documents ---

question = "What are 2 LangGraph agents used in production in 2024?"

# as an example retrieval_grader.invoke({"question": question, "document": doc_txt})
docs = retriever.invoke(question)

doc_txt = docs[0].page_content

result = retrieval_grader.invoke({"question": question, "document": doc_txt})

print("\n\nGrade Result: \n", result)


Results: 
 [Document(id='43c1d910-9305-4d42-b05a-77a928b41049', metadata={'source': 'https://blog.langchain.dev/top-5-langgraph-agents-in-production-2024/', 'title': 'Top 5 LangGraph Agents in Production 2024', 'language': 'en'}, page_content='Top 5 LangGraph Agents in Production 2024\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTop 5 LangGraph Agents in Production 2024\n\n3 min read\nDec 31, 2024'), Document(id='87e5c51e-ac00-4001-8054-161c0bd3aec2', metadata={'source': 'https://blog.langchain.dev/top-5-langgraph-agents-in-production-2024/', 'title': 'Top 5 LangGraph Agents in Production 2024', 'language': 'en'}, page_content='2024 was the year that agents started to work in production. Not the wide-ranging, fully autonomous agen

##### Create Dataset

In [9]:
from langsmith import wrappers, Client
from pydantic import BaseModel, Field
from openai import OpenAI

client = Client()
openai_client = wrappers.wrap_openai(OpenAI())

examples = [
    {
        "question": "Which companies are highlighted as top LangGraph agent adopters in 2024?",
        "answer": "The top adopters include Uber (code migration tools), AppFolio (property management copilot), LinkedIn (SQL Bot), Elastic (AI assistant), and Replit (multi-agent development platform) :cite[3]."
    },
    {
        "question": "How did AppFolio's AI copilot impact property managers?",
        "answer": "AppFolio's Realm-X AI copilot saved property managers over 10 hours per week by automating queries, bulk actions, and scheduling :cite[3]."
    },
    {
        "question": "What infrastructure trends dominated LLM usage in 2024?",
        "answer": "OpenAI remained the top LLM provider (6x more usage than Ollama), while open-source models via Ollama and Groq surged. Chroma and FAISS led vector stores, with MongoDB and Elastic gaining traction :cite[2]:cite[5]."
    },
    {
        "question": "How did LangGraph improve agent workflows compared to 2023?",
        "answer": "LangGraph usage grew to 43% of LangSmith organizations, with 21.9% of traces involving tool calls (up from 0.5% in 2023), enabling complex multi-step tasks like database writes :cite[2]:cite[7]."
    },
    {
        "question": "What distinguishes Replit's LangGraph implementation?",
        "answer": "Replit's agent emphasizes human-in-the-loop validation and a multi-agent architecture for code generation, combining autonomy with controlled outputs :cite[3]."
    }
]

inputs = [{"question": example["question"]} for example in examples]
outputs = [{"answer": example["answer"]} for example in examples]

# Programmatically create a dataset in LangSmith
dataset = client.create_dataset(
    dataset_name="langgraph-blogs-qa", description="Langchain blogs QA."
)

# Add examples to the dataset
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

print(
    f"Dataset created in langsmith with ID: {dataset.id}\n Navigate to {dataset.url}.")


Dataset created in langsmith with ID: 31147dab-5af7-48e1-8c42-c0079e0a3d8a
 Navigate to https://smith.langchain.com/o/131371ab-0bf2-4ad3-88cd-3b2d53ea1af9/datasets/31147dab-5af7-48e1-8c42-c0079e0a3d8a.
